First, we get the nessessary information on London Boroughs, dropping the extras, that will not be needed for the analysis.



In [3]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

LDF=pd.read_html(url, header=0)[0]

LDF.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


We filter out columns that are not needed in our analysis

In [4]:
LF = LDF.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
LF['Inner'].replace(np.nan,'0', inplace=True)
LF['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
LF['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
LF['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
LF.head()
LF['Inner'] = '0'
LF.head()

,Borough,Inner,Area (sq mi),Population (2019 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp..."
1,Barnet,0,33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


Then we rename the columns, making the dataset better on the eyes. Because of extra notes in the Wiki page, we will rename some of the Boroughs. Due to the staggering difference in rent price, as well as the amount of venues in London, we will filter to have only the Outer boroughs going forward

In [5]:
LF['Inner'] = LF.Borough.isin(Inn).astype(int)
Out = LF[LF.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
df = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2019 est)[1]":"Population"})
geolocator = Nominatim(user_agent="coursera-capstone-project")
df['Co-ordinates']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Co-ordinates'].apply(pd.Series)
df.head()

,Borough,Area,Population,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",39.969238,-82.936864
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814


In [130]:
df.shape[0]

20

Finally, we edit the coordinates, that are incorrect.

In [6]:
Rent = ['457.796','462.483','477.277','485.714','427.03','436.486','569.514','332.781','623.592','597.549','577.333','436.278','874.501','649.436','558.131','509.835','477.839','658.977','563.822','511.788']

df['Rent'] = Rent

df["Rent"] = pd.to_numeric(df["Rent"])

Long_list = df['Longitude'].tolist()
Lat_list = df['Latitude'].tolist()

In [92]:
df

,Borough,Area,Population,Co-ordinates,Latitude,Longitude,Rent
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504,457.796
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226,462.483
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",39.969238,-82.936864,477.277
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",32.937346,-87.164718,485.714
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814,427.030
6,Croydon,33.41,386710,"(51.3713049, -0.101957)",51.371305,-0.101957,436.486
7,Ealing,21.44,341806,"(51.5126553, -0.3051952)",51.512655,-0.305195,569.514
8,Enfield,31.74,333794,"(53.4308358, -2.9609095414165294)",53.430836,-2.960910,332.781
12,Haringey,11.42,268647,"(51.587929849999995, -0.10541010599099046)",51.587930,-0.105410,623.592
13,Harrow,19.49,251160,"(51.596827149999996, -0.3373046180437286)",51.596827,-0.337305,597.549


Coordinates Corrections for some boroughs

In [7]:
df.loc[df.Borough == "Brent", "Longitude"] = 0.2711
df.loc[df.Borough == "Brent", "Latitude"] = 51.5673
df.loc[df.Borough == "Bexley", "Longitude"] = 0.1543
df.loc[df.Borough == "Bexley", "Latitude"] = 51.4399
df.loc[df.Borough == "Sutton", "Longitude"] = 0.1940
df.loc[df.Borough == "Sutton", "Latitude"] = 51.3614
df.loc[df.Borough == "Enfield", "Longitude"] = 0.0807
df.loc[df.Borough == "Enfield", "Latitude"] = 51.6523

In [110]:
df

,Borough,Area,Population,Co-ordinates,Latitude,Longitude,Rent
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504,457.796
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226,462.483
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",51.439900,0.154300,477.277
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",51.567300,0.271100,485.714
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814,427.030
6,Croydon,33.41,386710,"(51.3713049, -0.101957)",51.371305,-0.101957,436.486
7,Ealing,21.44,341806,"(51.5126553, -0.3051952)",51.512655,-0.305195,569.514
8,Enfield,31.74,333794,"(53.4308358, -2.9609095414165294)",51.652300,0.080700,332.781
12,Haringey,11.42,268647,"(51.587929849999995, -0.10541010599099046)",51.587930,-0.105410,623.592
13,Harrow,19.49,251160,"(51.596827149999996, -0.3373046180437286)",51.596827,-0.337305,597.549


Now lets make a map, with all boroughs in question marked up



In [8]:
address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


In [170]:
!pip install folium
import folium

Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)


for lat, lng, label in zip(df['Latitude'], df['Longitude'], 
                            df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh

Now that we have a map, its time to get the venues from Foursquare. Due to a big borough area, we will take a rather big radius of search.



In [9]:
CLIENT_ID = 'FHEZFSKPIMIGOORDKAPCJHXA3C44PTVLENQSLRDTOAZDZJOW' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'UVTCNQPSXMTF4OFS00B5KDPDM3GSMKYNB4NCKYMBH1D4PKH2' #'your-client-secret' # your Foursquare Secret
VERSION = '20190915' # Foursquare API version

print('My credentails:')
print('My CLIENT_ID: ' + CLIENT_ID)
print('My CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
My CLIENT_ID: FHEZFSKPIMIGOORDKAPCJHXA3C44PTVLENQSLRDTOAZDZJOW
My CLIENT_SECRET:UVTCNQPSXMTF4OFS00B5KDPDM3GSMKYNB4NCKYMBH1D4PKH2


In [20]:

radius = 5000
LIMIT = 100

def getVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

In [11]:
Brgh_Venues = getVenues(names=df['Borough'],
                        latitudes=df['Latitude'],
                        longitudes=df['Longitude'])

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Croydon
Ealing
Enfield
Haringey
Harrow
Havering
Hillingdon
Hounslow
Kingston upon Thames
Merton
Newham
Redbridge
Richmond upon Thames
Sutton
Waltham Forest


Lets count the venues in every borough, to get the data scale



In [21]:
Brgh_Venues.groupby('Borough').count()


,Latitude,Longitude,Venue,Venue_Lat,Venue_Long,Venue_Category
Borough,,,,,,
Barking and Dagenham,76,76,76,76,76,76
Barnet,100,100,100,100,100,100
Bexley,100,100,100,100,100,100
Brent,43,43,43,43,43,43
Bromley,100,100,100,100,100,100
Croydon,100,100,100,100,100,100
Ealing,100,100,100,100,100,100
Enfield,40,40,40,40,40,40
Haringey,100,100,100,100,100,100


Lets break these numbers down to see which venues are more popular than the others



In [22]:
London_Brgh_onehot = pd.get_dummies(Brgh_Venues[['Venue_Category']], prefix="", prefix_sep="")
mid =  Brgh_Venues['Borough']

London_Brgh_onehot.insert(0, 'Borough', mid)

London_Brgh_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [23]:
Brgh_grouped = London_Brgh_onehot.groupby('Borough').mean().reset_index()
Brgh_grouped

,Borough,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Yoga Studio
0,Barking and Dagenham,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000,0.013158,0.00,0.00,0.000000,0.00,0.00,0.00,0.00
1,Barnet,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000,0.030000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00
2,Bexley,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000,0.000000,0.00,0.00,0.010000,0.00,0.00,0.00,0.00
3,Brent,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000,0.023256,0.00,0.00,0.000000,0.00,0.00,0.00,0.00
4,Bromley,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,...,0.01,0.000,0.020000,0.00,0.00,0.000000,0.00,0.00,0.00,0.01
5,Croydon,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.000000,...,0.01,0.000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00
6,Ealing,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.000000,...,0.00,0.000,0.000000,0.00,0.01,0.000000,0.00,0.00,0.00,0.00
7,Enfield,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.025,0.025000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00
8,Haringey,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000,0.050000,0.01,0.00,0.000000,0.00,0.01,0.01,0.00
9,Harrow,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.000,0.010000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00


Below there is the complete list of venues in this area

In [192]:
for col_name in Brgh_grouped.columns: 
    print(col_name)

Borough
Afghan Restaurant
African Restaurant
American Restaurant
Arcade
Argentinian Restaurant
Art Gallery
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
Auto Garage
Auto Workshop
Bakery
Bar
Beer Bar
Beer Garden
Beer Store
Bistro
Bookstore
Botanical Garden
Brasserie
Brazilian Restaurant
Breakfast Spot
Brewery
Burger Joint
Burrito Place
Bus Stop
Café
Cajun / Creole Restaurant
Canal
Canal Lock
Caribbean Restaurant
Cave
Cheese Shop
Chinese Restaurant
Chocolate Shop
Church
Climbing Gym
Clothing Store
Cocktail Bar
Coffee Shop
Comfort Food Restaurant
Concert Hall
Convenience Store
Cosmetics Shop
Coworking Space
Creperie
Cricket Ground
Cupcake Shop
Cycle Studio
Dam
Deli / Bodega
Department Store
Dessert Shop
Dim Sum Restaurant
Diner
Discount Store
Distillery
Donut Shop
Eastern European Restaurant
Electronics Store
English Restaurant
Escape Room
Event Space
Farm
Farmers Market
Fast Food Restaurant
Film Studio
Fish & Chips Shop
Fish Market
Flea Market
Food
Food & Drink Shop
Food Court


Lets see, which of the venues are the most popular?

In [24]:
num_top_venues = 10

for brgh in Brgh_grouped['Borough']:
    print("_________"+brgh+"________")
    temp = Brgh_grouped[Brgh_grouped['Borough'] == brgh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

_________Barking and Dagenham________
                  venue  freq
0           Supermarket  0.12
1         Grocery Store  0.12
2           Gas Station  0.07
3                  Park  0.07
4                   Pub  0.07
5           Coffee Shop  0.07
6    Italian Restaurant  0.04
7  Fast Food Restaurant  0.04
8           Pizza Place  0.04
9                   Gym  0.03


_________Barnet________
                venue  freq
0                 Pub  0.10
1       Grocery Store  0.09
2         Coffee Shop  0.09
3                Café  0.06
4  Italian Restaurant  0.05
5         Supermarket  0.05
6         Pizza Place  0.04
7      Sandwich Place  0.04
8               Hotel  0.04
9  Turkish Restaurant  0.03


_________Bexley________
                    venue  freq
0                     Pub  0.17
1           Grocery Store  0.11
2             Coffee Shop  0.10
3             Pizza Place  0.07
4                   Hotel  0.06
5             Supermarket  0.06
6          Clothing Store  0.05
7               

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
brgh_venues_sorted = pd.DataFrame(columns=columns)
brgh_venues_sorted['Borough'] = Brgh_grouped['Borough']

for ind in np.arange(Brgh_grouped.shape[0]):
    brgh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brgh_grouped.iloc[ind, :], num_top_venues)

brgh_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Supermarket,Grocery Store,Gas Station,Park,Coffee Shop,Pub,Italian Restaurant,Fast Food Restaurant,Pizza Place,Hotel
1,Barnet,Pub,Grocery Store,Coffee Shop,Café,Supermarket,Italian Restaurant,Pizza Place,Sandwich Place,Hotel,Park
2,Bexley,Pub,Grocery Store,Coffee Shop,Pizza Place,Hotel,Supermarket,Clothing Store,Park,Pharmacy,Furniture / Home Store
3,Brent,Pub,Hotel,Grocery Store,Italian Restaurant,Coffee Shop,Sandwich Place,Golf Course,Supermarket,Convenience Store,Chinese Restaurant
4,Bromley,Grocery Store,Pub,Coffee Shop,Pizza Place,Gym / Fitness Center,Indian Restaurant,Park,Clothing Store,Supermarket,Italian Restaurant
5,Croydon,Grocery Store,Pub,Coffee Shop,Park,Hotel,Mediterranean Restaurant,Pharmacy,Sandwich Place,Supermarket,Pizza Place
6,Ealing,Pub,Coffee Shop,Park,Café,Botanical Garden,Pizza Place,Gym / Fitness Center,Gastropub,Italian Restaurant,Hotel
7,Enfield,Pub,Café,Hotel,Grocery Store,Italian Restaurant,Steakhouse,Intersection,Pharmacy,Clothing Store,Portuguese Restaurant
8,Haringey,Pub,Café,Coffee Shop,Park,Turkish Restaurant,Bakery,Italian Restaurant,Pizza Place,Trail,Climbing Gym
9,Harrow,Coffee Shop,Indian Restaurant,Sandwich Place,Grocery Store,Pub,Gym / Fitness Center,Pharmacy,Park,Supermarket,Café


By testing the following code with different number of clusters we conclude that two clusters is the most suitable number for our analysis.

In [27]:
kclusters = 3

brgh_grouped_clustering = Brgh_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brgh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0])

Lets join all of our dataframes together

In [28]:
#brgh_venues_sorted = brgh_venues_sorted.drop("Cluster Label", axis=1)
brgh_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Borough_merged = pd.merge(df,brgh_venues_sorted, on='Borough')
Borough_merged

,Borough,Area,Population,Co-ordinates,Latitude,Longitude,Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,13.93,212906,"(51.5541171, 0.15050434261994267)",51.554117,0.150504,457.796,1,Supermarket,Grocery Store,Gas Station,Park,Coffee Shop,Pub,Italian Restaurant,Fast Food Restaurant,Pizza Place,Hotel
1,Barnet,33.49,395896,"(51.65309, -0.2002261)",51.653090,-0.200226,462.483,1,Pub,Grocery Store,Coffee Shop,Café,Supermarket,Italian Restaurant,Pizza Place,Sandwich Place,Hotel,Park
2,Bexley,23.38,248287,"(39.9692378, -82.936864)",51.439900,0.154300,477.277,1,Pub,Grocery Store,Coffee Shop,Pizza Place,Hotel,Supermarket,Clothing Store,Park,Pharmacy,Furniture / Home Store
3,Brent,16.70,329771,"(32.9373463, -87.1647184)",51.567300,0.271100,485.714,1,Pub,Hotel,Grocery Store,Italian Restaurant,Coffee Shop,Sandwich Place,Golf Course,Supermarket,Convenience Store,Chinese Restaurant
4,Bromley,57.97,332336,"(51.4028046, 0.0148142)",51.402805,0.014814,427.030,1,Grocery Store,Pub,Coffee Shop,Pizza Place,Gym / Fitness Center,Indian Restaurant,Park,Clothing Store,Supermarket,Italian Restaurant
5,Croydon,33.41,386710,"(51.3713049, -0.101957)",51.371305,-0.101957,436.486,1,Grocery Store,Pub,Coffee Shop,Park,Hotel,Mediterranean Restaurant,Pharmacy,Sandwich Place,Supermarket,Pizza Place
6,Ealing,21.44,341806,"(51.5126553, -0.3051952)",51.512655,-0.305195,569.514,0,Pub,Coffee Shop,Park,Café,Botanical Garden,Pizza Place,Gym / Fitness Center,Gastropub,Italian Restaurant,Hotel
7,Enfield,31.74,333794,"(53.4308358, -2.9609095414165294)",51.652300,0.080700,332.781,0,Pub,Café,Hotel,Grocery Store,Italian Restaurant,Steakhouse,Intersection,Pharmacy,Clothing Store,Portuguese Restaurant
8,Haringey,11.42,268647,"(51.587929849999995, -0.10541010599099046)",51.587930,-0.105410,623.592,0,Pub,Café,Coffee Shop,Park,Turkish Restaurant,Bakery,Italian Restaurant,Pizza Place,Trail,Climbing Gym
9,Harrow,19.49,251160,"(51.596827149999996, -0.3373046180437286)",51.596827,-0.337305,597.549,1,Coffee Shop,Indian Restaurant,Sandwich Place,Grocery Store,Pub,Gym / Fitness Center,Pharmacy,Park,Supermarket,Café


Mapping the clusters

In [29]:
import folium
map_clusters = folium.Map(location=[London_latitude, London_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, rent, pop in zip(Borough_merged['Latitude'],
                                  Borough_merged['Longitude'],
                                  Borough_merged['Borough'],
                                  Borough_merged['Cluster Label'],
                                  Borough_merged['Rent'],
                                  Borough_merged['Population']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + " " + "Rent " + str(rent) + " " + "Population " + str(pop), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

After seeing the clusters, it is worth looking into each one with a bit more detail, to see, which one is better suited for opening a venue.
The following is the first cluster.


In [204]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 0, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]


,Borough,Area,Population,Latitude,Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Ealing,21.44,341806,51.512655,569.514,0,Pub,Coffee Shop,Park,Café,Botanical Garden,Pizza Place,Gym / Fitness Center,Gastropub,Italian Restaurant,Hotel
7,Enfield,31.74,333794,51.652300,332.781,0,Pub,Café,Hotel,Grocery Store,Italian Restaurant,Steakhouse,Intersection,Pharmacy,Clothing Store,Portuguese Restaurant
8,Haringey,11.42,268647,51.587930,623.592,0,Pub,Café,Coffee Shop,Park,Turkish Restaurant,Bakery,Italian Restaurant,Pizza Place,Trail,Climbing Gym
10,Havering,43.35,259552,51.544002,577.333,0,Bakery,Hotel,Pub,Coffee Shop,Park,Cocktail Bar,Garden,Pizza Place,Bookstore,Department Store
12,Hounslow,21.61,271523,51.468613,874.501,0,Pub,Park,Indian Restaurant,Garden,Coffee Shop,Café,Pizza Place,Restaurant,Hotel,Gastropub
13,Kingston upon Thames,14.38,177507,51.409627,649.436,0,Pub,Park,Café,Coffee Shop,Gastropub,Supermarket,Italian Restaurant,Garden,Thai Restaurant,Scenic Lookout
14,Merton,14.52,206548,51.410870,558.131,0,Pub,Coffee Shop,Park,Café,Sushi Restaurant,Italian Restaurant,Lebanese Restaurant,French Restaurant,Tennis Stadium,Pizza Place
15,Newham,13.98,353134,51.530000,509.835,0,Pub,Park,Bar,Italian Restaurant,Coffee Shop,Restaurant,Café,Beer Bar,Canal Lock,English Restaurant
16,Redbridge,21.78,305222,51.576320,477.839,0,Pub,Park,Coffee Shop,Gym / Fitness Center,Café,Italian Restaurant,Grocery Store,Restaurant,Pizza Place,Bakery
17,Richmond upon Thames,22.17,198019,51.440553,658.977,0,Pub,Park,Café,Garden,Coffee Shop,Italian Restaurant,German Restaurant,Botanical Garden,Gastropub,Hotel


This is the second cluster

In [186]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 1, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]


,Borough,Area,Population,Latitude,Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Barking and Dagenham,13.93,212906,51.554117,457.796,1,Supermarket,Grocery Store,Gas Station,Park,Coffee Shop,Pub
1,Barnet,33.49,395896,51.653090,462.483,1,Pub,Grocery Store,Coffee Shop,Café,Supermarket,Italian Restaurant
2,Bexley,23.38,248287,51.439900,477.277,1,Pub,Grocery Store,Coffee Shop,Pizza Place,Hotel,Supermarket
3,Brent,16.70,329771,51.567300,485.714,1,Pub,Hotel,Grocery Store,Italian Restaurant,Coffee Shop,Sandwich Place
4,Bromley,57.97,332336,51.402805,427.030,1,Grocery Store,Pub,Coffee Shop,Pizza Place,Gym / Fitness Center,Indian Restaurant
5,Croydon,33.41,386710,51.371305,436.486,1,Grocery Store,Pub,Coffee Shop,Park,Hotel,Mediterranean Restaurant
9,Harrow,19.49,251160,51.596827,597.549,1,Coffee Shop,Indian Restaurant,Sandwich Place,Grocery Store,Pub,Gym / Fitness Center
11,Hillingdon,44.67,306870,51.542519,436.278,1,Coffee Shop,Pub,Supermarket,Indian Restaurant,Sandwich Place,Clothing Store


This is the 3rd cluster

In [187]:
Borough_merged.loc[Borough_merged['Cluster Label'] == 2, Borough_merged.columns[[0,1,2,4] + list(range(6, Borough_merged.shape[1]))]]


,Borough,Area,Population,Latitude,Rent,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
18,Sutton,16.93,206349,51.3614,563.822,2,Pub,Golf Course,Racetrack,Train Station,Bar,Historic Site
